# keras模型保存和序列化

- [1. 准备数据和定义模型](#1.准备数据和定义模型)

- [2. 以checkpoints形式保存](#2.以checkpoints形式保存)

    - [2.1 加载模型进行预测](#2.1加载模型进行预测)
    
    - [2.2 调整保存相关参数](#2.2调整保存相关参数)
    
- []()


## 1.准备数据和定义模型

In [1]:
#!pip install -q pyyaml h5py  # 需要以 HDF5 格式保存模型
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

# 准备数据集
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:-1000]
test_labels = test_labels[-1000:]

train_images = train_images[:-1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[-1000:].reshape(-1, 28 * 28) / 255.0

# 定义模型
# 定义一个简单的序列模型
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

# 创建一个基本的模型实例
model = create_model()

# 显示模型的结构
model.summary()

2.0.0-beta1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 2.以checkpoints形式保存

可以使用训练好的模型而无需从头开始重新训练，**或在打断的地方开始训练**，以防止训练过程没有保存。 

`tf.keras.callbacks.ModelCheckpoint` 允许在训练的过程中和结束时回调保存的模型。

[回到目录](#keras模型保存和序列化)

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 使用新的回调训练模型
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 通过回调训练

W0115 16:40:43.202754 139836618565440 deprecation.py:323] From /usr/local/python3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 59000 samples, validate on 1000 samples
Epoch 1/10
58624/59000 [============================>.] - ETA: 0s - loss: 0.2254 - accuracy: 0.9331
Epoch 00001: saving model to training_1/cp.ckpt
59000/59000 [==============================] - 5s 89us/sample - loss: 0.2247 - accuracy: 0.9333 - val_loss: 0.1058 - val_accuracy: 0.9680
Epoch 2/10
58752/59000 [============================>.] - ETA: 0s - loss: 0.0983 - accuracy: 0.9701
Epoch 00002: saving model to training_1/cp.ckpt
59000/59000 [==============================] - 5s 81us/sample - loss: 0.0982 - accuracy: 0.9701 - val_loss: 0.1080 - val_accuracy: 0.9710
Epoch 3/10
58368/59000 [============================>.] - ETA: 0s - loss: 0.0694 - accuracy: 0.9788
Epoch 00003: saving model to training_1/cp.ckpt
59000/59000 [==============================] - 5s 79us/sample - loss: 0.0694 - accuracy: 0.9788 - val_loss: 0.0813 - val_accuracy: 0.9760
Epoch 4/10
58816/59000 [============================>.] - ETA: 0s - loss: 0.0542 - accuracy: 

In [5]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


### 2.1加载模型进行预测

仅恢复模型的权重时，**必须具有与原始模型具有相同网络结构的模型**。由于模型具有相同的结构，您可以共享权重，尽管它是模型的不同实例。

[回到目录](#keras模型保存和序列化)

In [8]:
# 创建一个基本模型实例
model = create_model()

# 评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

# 加载权重
model.load_weights(checkpoint_path)

# 重新评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.4252 - accuracy: 0.0800
Untrained model, accuracy:  8.00%
1000/1000 - 0s - loss: 0.0824 - accuracy: 0.9820
Restored model, accuracy: 98.20%


### 2.2调整保存相关参数

- 为 checkpoint 提供唯一名称并调整 checkpoint 频率

- 每五个 epochs 保存一次唯一命名的 checkpoint 

[回到目录](#keras模型保存和序列化)

In [9]:
# 在文件名中包含 epoch (使用 `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，每 5 个 epochs 保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 创建一个新的模型实例
model = create_model()

# 使用 `checkpoint_path` 格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回调*训练*模型
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)

W0115 17:01:31.837317 139836618565440 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W0115 17:01:31.977887 139836618565440 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0115 17:01:31.978780 139836618565440 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0115 17:01:31.979504 139836618565440 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0115 17:01:31.980298 139836618565440 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0115 17:01:31.981003 139836618565440 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0115 17:01:31.983219 139836618565440 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.re


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# 创建一个新的模型实例
model = create_model()

# 加载以前保存的权重
model.load_weights(latest)

# 重新评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.1521 - accuracy: 0.9840
Restored model, accuracy: 98.40%


上述代码将权重存储到 checkpoint 格式化文件的集合中，这些文件仅包含二进制格式的训练权重。

Checkpoints 包含：

- 一个或多个包含模型权重的分片。
- 索引文件，指示哪些权重存储在哪个分片中。

**如果你只在一台机器上训练一个模型，你将有一个带有后缀的碎片： .data-00000-of-00001**